In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import torch
import numpy as np

from gnnboundary import *

### Load Dataset

In [ ]:
seed = 12345
dataset_name = "motif"

datasets = {
 "collab": CollabDataset(seed=seed),
 "motif": MotifDataset(seed=seed),
 "enzymes": ENZYMESDataset(seed=seed)
}

dataset = datasets[dataset_name]

### Load Model

In [ ]:
config = {
    "node_features": len(dataset.NODE_CLS),
    "num_classes": len(dataset.GRAPH_CLS),
}

match dataset_name:
    case "collab":
        config["hidden_channels"] = 64
        config["num_layers"] = 5
    case "motif":
        config["hidden_channels"] = 6
        config["num_layers"] = 3
    case "enzymes":
        config["hidden_channels"] = 32
        config["num_layers"] = 3
    case _:
        pass

model = GCNClassifier(**config)
model.load_state_dict(torch.load(f"ckpts/{dataset_name}_retrained.pt")['model'])

In [ ]:
dataset_list_gt = dataset.split_by_class()
dataset_list_pred = dataset.split_by_pred(model)

### Evaluate Model

In [ ]:
evaluation = dataset.model_evaluate(model, use_test_only=False)
evaluation

In [ ]:
draw_matrix(
    evaluation['cm'], 
    dataset.GRAPH_CLS.values(), 
    fmt='d', 
    file_name=f"{dataset_name}_confusion_all.jpg", 
    save_path=".\\figures\\confusion_matrix\\from_checkpoints")

### Compute adjacency

In [ ]:
from tqdm import tqdm

experiment_results = []

for _ in tqdm(range(10)):
    adj_ratio_mat, _ = pairwise_boundary_analysis(model, dataset_list_pred)
    experiment_results.append(adj_ratio_mat)

result = np.array(experiment_results).mean(axis=0)
draw_matrix(
    result, 
    names=dataset.GRAPH_CLS.values(), 
    fmt='.2f',
    file_name=f"{dataset_name}_adjacency.jpg", 
    save_path=".\\figures\\adjacency_matrix\\from_checkpoints")